<a href="https://colab.research.google.com/github/Theod0reWu/AdventOfCode/blob/main/5_conv2d_dw_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Depthwise-seperable 2D Convolution on GPU

## 1. Set-up

In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Make sure your token is stored in a txt file at the location below.
# This way there is no risk that you will push it to your repo
# Never share your token with anyone, it is basically your github password!
with open('/content/gdrive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username
with open('/content/gdrive/MyDrive/ece5545/git_username.txt') as f:
    handle = f.readline().strip()

In [3]:
# Clone your github repo
YOUR_TOKEN = token
YOUR_HANDLE = handle
BRANCH = "main"

%mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545
!git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a3-{YOUR_HANDLE}.git
%cd /content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}
!git checkout {BRANCH}
!git pull
%cd /content/gdrive/MyDrive/ece5545

PROJECT_ROOT = f"/content/gdrive/MyDrive/ece5545/a3-{YOUR_HANDLE}"

mkdir: cannot create directory ‘/content/gdrive/MyDrive/ece5545’: File exists
/content/gdrive/MyDrive/ece5545
fatal: destination path 'a3-Theod0reWu' already exists and is not an empty directory.
/content/gdrive/MyDrive/ece5545/a3-Theod0reWu
M	1-conv1d_cpu.ipynb
M	2-conv1d_gpu.ipynb
M	2_conv1d_gpu.ipynb
M	4-gemm_gpu.ipynb
M	5-conv2d_dw_gpu.ipynb
M	src/ops.py
Already on 'main'
Your branch is behind 'origin/main' by 5 commits, and can be fast-forwarded.
  (use "git pull" to update your local branch)
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 7 (delta 2), reused 4 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (7/7), 5.62 KiB | 6.00 KiB/s, done.
From https://github.com/ML-HW-SYS/a3-Theod0reWu
   72c0703..99a7b04  main       -> origin/main
Updating 3b5b20a..99a7b04
error: Your local changes to the following files would be overwritten by merge:
	src/ops.py
Please commit your change

In [4]:
# This extension reloads all imports before running each cell
%load_ext autoreload
%autoreload 2

In [5]:
!ls {PROJECT_ROOT}

1-conv1d_cpu.ipynb  2_conv1d_gpu.ipynb	 4-gemm_gpu.ipynb	leaderboard_id.txt  src
2-conv1d_gpu.ipynb  3-conv1d_fpga.ipynb  5-conv2d_dw_gpu.ipynb	README.md	    tests


## 2 Install TVM

In [6]:
!pip install numpy==1.24.3
!pip install tlcpack-nightly-cu102 -f https://tlcpack.ai/wheels

Looking in links: https://tlcpack.ai/wheels
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.5/428.5 MB 4.3 MB/s eta 0:00:00


In [24]:
from scipy.signal import convolve2d
def numpy_depthwise_conv2d(input_np, kernel_np, padding='same', strides=(1, 1)):
    """
    Performs depthwise 2D convolution using scipy.signal.convolve2d.

    Args:
        input_np (np.ndarray): Input tensor of shape (B, C, H, W).
        kernel_np (np.ndarray): Depthwise kernel of shape (C, 1, KH, KW).
        padding (str or int): Padding mode ('valid', 'same', or integer).
        strides (tuple): Strides for height and width.

    Returns:
        np.ndarray: Output tensor of shape (B, C, out_H, out_W).
    """
    batch_size, in_channels, in_height, in_width = input_np.shape
    out_channels, _, kernel_height, kernel_width = kernel_np.shape
    stride_h, stride_w = strides

    output_np = np.zeros_like(input_np, dtype=input_np.dtype)

    for b in range(batch_size):
        for c in range(in_channels):
            kernel_slice = kernel_np[c, 0, :, :]
            input_slice = input_np[b, c, :, :]
            convolved_slice = convolve2d(input_slice, kernel_slice, mode=padding)
            output_np[b, c, :, :] = convolved_slice[::stride_h, ::stride_w]
    return output_np

## 3. Implement `make_dwsp_conv2d_gpu_scheduler` function in `src.ops`

In that function, you are required to implemented 2D convolution and use TVM to optimize it.
Please use zero padding and unit stride.
You can assume kernel size to be an odd number.
The padding will equals to kernel size minus ones.
In this case, the output image will preserve the input image dimension.

The `make_dwsp_conv2d_gpu_scheduler` takes following arguments:
1. Batch size $B$;
2. Input channel size $C$;
3. Input image height $H$;
4. Input image width $W$;
5. Output number of channels $O$;
6. Kernel size $K$

You should return both the TVM scheduler and the TVM opterator for
1. Input tensor $x$ with size (B, C, H, W)
2. Input kernel weight $y$ with size (O, 1, K, K)
3. Output $out$ with size (B, O, H, W)

The scheduler should be able to used to build a function with signature $func(x, y, out)$.
Please see the following cells the usage.

In [37]:
import tvm
import torch.nn.functional as F
import numpy as np
import sys
# Adding assignment 3 to the system path
# Make sure this matches your git directory
sys.path.insert(0, PROJECT_ROOT)
from src.ops import make_dwsp_conv2d_gpu_scheduler

B = 3
C = 4
H = 16
W = 32
K = 7
dtype = 'float32'
a_np = np.random.rand(B, C, H, W).astype(dtype)
w_np = np.random.rand(C, 1, K, K).astype(dtype)
ans = numpy_depthwise_conv2d(a_np, w_np)

s, inp, ker, out = make_dwsp_conv2d_gpu_scheduler(B, C, H, W, K)
func = tvm.build(s, [inp, ker, out], "cuda")

dev = tvm.cuda(0)
a = tvm.nd.array(a_np, dev)
w = tvm.nd.array(w_np, dev)
b = tvm.nd.array(np.zeros((B, C, H, W), dtype), dev)
func(a, w, b)
evaluator = func.time_evaluator(func.entry_name, dev, number=1, repeat =1)

print("Output:", np.sum(b.asnumpy()))
print("Answer:", np.sum(ans))
print(f"2DConv TVM: %f ms" % (evaluator(a, w, b).mean * 1e3))

Output: 62191.125
Answer: 62151.516
2DConv TVM: 0.020255 ms


In [38]:
print(tvm.lower(s, [inp, ker, out], simple_mode=True))

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((3, 4, 16, 32), "float32"), W: T.Buffer((4, 1, 7, 7), "float32"), out: T.Buffer((3, 4, 16, 32), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        blockIdx_y = T.launch_thread("blockIdx.y", 4)
        threadIdx_y = T.launch_thread("threadIdx.y", 1)
        blockIdx_z = T.launch_thread("blockIdx.z", 32)
        threadIdx_z = T.launch_thread("threadIdx.z", 1)
        blockIdx_x = T.launch_thread("blockIdx.x", 32)
        threadIdx_x = T.launch_thread("threadIdx.x", 16)
        out_1 = T.Buffer((6144,), data=out.data)
        if T.likely(blockIdx_y < 3):
            if T.likely(blockIdx_z < 4):
                out_1[blockIdx_y * 2048 + blockIdx_z * 512 + blockIdx_x * 16 + threadIdx_x] = T.float32(0)
        for dh, dw in T.grid(7, 7):
            if T.likely(blockIdx_y < 3):
                if

In [39]:
%cd {PROJECT_ROOT}
!python -m pytest tests/test_dwsp_2dconv_gpu.py

/content/gdrive/MyDrive/ece5545/a3-Theod0reWu
======================================= test session starts ========================================
platform linux -- Python 3.11.11, pytest-8.3.5, pluggy-1.5.0
rootdir: /content/gdrive/MyDrive/ece5545/a3-Theod0reWu
plugins: anyio-4.9.0, langsmith-0.3.22, typeguard-4.4.2
collected 1357 items                                                                               

tests/test_dwsp_2dconv_gpu.py .............................................................. [  4%]
............................................................................................ [ 11%]
............................................................................................ [ 18%]
............................................................................................ [ 24%]
.......................................................................................FF... [ 31%]
..FF.....FF....FFF..............................................................